In [1]:
"""
Subway Surfers - style endless runner (single-file)
Author: ChatGPT (example)
Language: Python 3 + pygame
Notes:
 - Designed to be runnable on Replit or locally.
 - No external images required (uses simple shapes/colors).
 - Controls:
    - Left / Right arrows : change lanes
    - Up arrow or Space   : Jump
    - Down arrow         : Roll / Duck
    - H                  : Toggle hoverboard (if purchased in-game)
    - R                  : Revive if you have keys (press when game over)
 - This is intended as a learning/starting implementation, not a full commercial clone.
"""

import pygame
import random
import math
import os
from pathlib import Path

# ---------- Configuration ----------
SCREEN_WIDTH = 480
SCREEN_HEIGHT = 800
FPS = 60

LANE_COUNT = 3
LANE_X = [SCREEN_WIDTH * 0.2, SCREEN_WIDTH * 0.5, SCREEN_WIDTH * 0.8]  # center x positions for lanes
GROUND_Y = SCREEN_HEIGHT - 140

PLAYER_WIDTH = 40
PLAYER_HEIGHT = 70
ROLL_HEIGHT = 40

OBSTACLE_WIDTH = 60
OBSTACLE_HEIGHT = 60

COIN_SIZE = 18

BASE_SPEED = 5.0
SPEED_INCREASE_PER_100 = 0.2  # speed increase every 100 distance units

# Powerup durations (in seconds)
MAGNET_DURATION = 7
JETPACK_DURATION = 5
SNEAKERS_DURATION = 6
MULTIPLIER_DURATION = 6
HOVERBOARD_DURATION = 10

# ---------- Utilities ----------
def load_highscore(path='highscore.txt'):
    try:
        with open(path, 'r') as f:
            return int(f.read().strip() or 0)
    except:
        return 0

def save_highscore(score, path='highscore.txt'):
    try:
        prev = load_highscore(path)
        if score > prev:
            with open(path, 'w') as f:
                f.write(str(score))
    except Exception as e:
        print("Couldn't save highscore:", e)

# ---------- Game classes ----------
class Player:
    def _init_(self):
        self.lane = 1  # 0,1,2
        self.x = LANE_X[self.lane]
        self.y = GROUND_Y - PLAYER_HEIGHT
        self.width = PLAYER_WIDTH
        self.height = PLAYER_HEIGHT
        self.color = (30, 144, 255)
        self.target_x = self.x
        self.vy = 0.0
        self.on_ground = True
        self.is_rolling = False
        self.gravity = 0.9
        self.jump_strength = -18
        self.speed_lerp = 0.35  # how fast to move sideways
        self.hoverboard = False
        self.hoverboard_timer = 0.0

    def rect(self):
        h = self.height if not self.is_rolling else ROLL_HEIGHT
        return pygame.Rect(int(self.x - self.width/2), int(self.y), self.width, h)

    def update(self, dt):
        # smooth lane movement
        self.x += (self.target_x - self.x) * self.speed_lerp

        # vertical physics
        if not self.on_ground:
            self.vy += self.gravity
            self.y += self.vy
            if self.y >= GROUND_Y - (self.height if not self.is_rolling else ROLL_HEIGHT):
                self.y = GROUND_Y - (self.height if not self.is_rolling else ROLL_HEIGHT)
                self.vy = 0
                self.on_ground = True

        # hoverboard timer
        if self.hoverboard:
            self.hoverboard_timer -= dt
            if self.hoverboard_timer <= 0:
                self.hoverboard = False

    def move_left(self):
        self.lane = max(0, self.lane - 1)
        self.target_x = LANE_X[self.lane]

    def move_right(self):
        self.lane = min(LANE_COUNT-1, self.lane + 1)
        self.target_x = LANE_X[self.lane]

    def jump(self):
        if self.on_ground:
            self.on_ground = False
            self.vy = self.jump_strength
            self.is_rolling = False

    def roll(self):
        if self.on_ground:
            self.is_rolling = True
            # adjust y so collider height matches roll height
            self.y = GROUND_Y - ROLL_HEIGHT

    def stop_roll(self):
        if self.is_rolling:
            self.is_rolling = False
            self.y = GROUND_Y - self.height

    def enable_hoverboard(self, duration):
        self.hoverboard = True
        self.hoverboard_timer = duration

class Obstacle:
    def _init_(self, lane, y, typ='train'):
        self.lane = lane
        self.x = LANE_X[lane]
        self.y = y
        self.typ = typ  # 'train', 'barrier'
        self.passed = False

    def rect(self):
        if self.typ == 'train':
            w = OBSTACLE_WIDTH * 1.4
            h = OBSTACLE_HEIGHT * 0.9
        else:
            w = OBSTACLE_WIDTH
            h = OBSTACLE_HEIGHT
        return pygame.Rect(int(self.x - w/2), int(self.y), int(w), int(h))

    def draw(self, surf):
        r = self.rect()
        if self.typ == 'train':
            pygame.draw.rect(surf, (139,69,19), r)  # brown train
            pygame.draw.rect(surf, (0,0,0), r, 2)
        else:
            pygame.draw.rect(surf, (220,20,60), r)  # red barrier
            pygame.draw.rect(surf, (0,0,0), r, 2)

class Coin:
    def _init_(self, lane, y):
        self.lane = lane
        self.x = LANE_X[lane]
        self.y = y
        self.size = COIN_SIZE
        self.collected = False

    def rect(self):
        return pygame.Rect(int(self.x - self.size/2), int(self.y), self.size, self.size)

    def draw(self, surf):
        r = self.rect()
        pygame.draw.ellipse(surf, (255,215,0), r)

class Powerup:
    def _init_(self, lane, y, kind):
        self.lane = lane
        self.x = LANE_X[lane]
        self.y = y
        self.kind = kind  # 'magnet','jetpack','sneakers','mult'
        self.size = 28

    def rect(self):
        return pygame.Rect(int(self.x - self.size/2), int(self.y), self.size, self.size)

    def draw(self, surf):
        r = self.rect()
        color_map = {'magnet':(178,34,34),'jetpack':(70,130,180),'sneakers':(34,139,34),'mult':(128,0,128)}
        pygame.draw.rect(surf, color_map.get(self.kind,(200,200,200)), r)
        pygame.draw.rect(surf, (0,0,0), r, 2)

# ---------- Main Game ----------
def main():
    pygame.init()
    screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
    pygame.display.set_caption("Mini Subway Surfers - Demo")
    clock = pygame.time.Clock()
    font = pygame.font.SysFont("Arial", 20)

    player = Player()
    obstacles = []
    coins = []
    powerups = []

    distance = 0.0
    score = 0
    coin_count = 0
    key_count = 1  # give one key to demo revive
    multiplier = 1
    multiplier_timer = 0.0

    magnet_active = False
    magnet_timer = 0.0
    jetpack_active = False
    jetpack_timer = 0.0
    sneakers_active = False
    sneakers_timer = 0.0

    game_over = False
    spawn_timer = 0.0
    spawn_interval = 1.0  # spawn obstacle every 1s initially
    speed = BASE_SPEED

    highscore = load_highscore()

    # helper to spawn obstacle/coin/powerup
    def spawn_segment(y_pos):
        r = random.random()
        lane = random.randrange(0, LANE_COUNT)
        if r < 0.6:
            # obstacle
            obstacles.append(Obstacle(lane, y_pos, typ='train' if random.random() < 0.7 else 'barrier'))
        elif r < 0.9:
            coins.append(Coin(lane, y_pos))
        else:
            kind = random.choice(['magnet','jetpack','sneakers','mult'])
            powerups.append(Powerup(lane, y_pos, kind))

    # initial content
    for i in range(6):
        spawn_segment(-i * 160)

    # main loop
    running = True
    while running:
        dt = clock.tick(FPS) / 1000.0  # seconds elapsed
        if not game_over:
            distance += speed * dt * 20  # scale to a nicer number
            score = int(distance) * multiplier + coin_count * 5

            # speed increases with distance
            speed = BASE_SPEED + (distance // 100) * (SPEED_INCREASE_PER_100)

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

            if event.type == pygame.KEYDOWN:
                if not game_over:
                    if event.key == pygame.K_LEFT:
                        player.move_left()
                    elif event.key == pygame.K_RIGHT:
                        player.move_right()
                    elif event.key in (pygame.K_UP, pygame.K_SPACE):
                        player.jump()
                    elif event.key == pygame.K_DOWN:
                        player.roll()
                    elif event.key == pygame.K_h:  # debug hoverboard
                        player.enable_hoverboard(HOVERBOARD_DURATION)
                else:
                    # game over keys
                    if event.key == pygame.K_r and key_count > 0:
                        # revive: simple revive that resets some params
                        key_count -= 1
                        game_over = False
                        # give player a short hoverboard to avoid instant death
                        player.enable_hoverboard(3)
                        player.x = LANE_X[1]
                        player.lane = 1
                        player.target_x = player.x
                        player.y = GROUND_Y - player.height
                        player.on_ground = True
                        obstacles.clear()
                        coins.clear()
                        powerups.clear()
                        distance = 0
                        speed = BASE_SPEED
                        for i in range(5):
                            spawn_segment(-i * 160)
                    elif event.key == pygame.K_q:
                        running = False

            if event.type == pygame.KEYUP:
                if event.key == pygame.K_DOWN:
                    player.stop_roll()

        # Spawn new obstacles / coins / powerups as world scrolls
        if not game_over:
            spawn_timer -= dt
            if spawn_timer <= 0:
                spawn_timer = max(0.45, spawn_interval - (distance/10000))  # spawn faster as distance increases
                spawn_segment(-160)

        # Move objects down the screen to simulate player moving forward
        remove_obstacles = []
        for ob in obstacles:
            ob.y += speed * 100 * dt
            # mark passed for scoring or removal
            if ob.y > SCREEN_HEIGHT + 100:
                remove_obstacles.append(ob)

        for c in coins:
            c.y += speed * 100 * dt
        for p in powerups:
            p.y += speed * 100 * dt

        # collision checks
        if not game_over:
            p_rect = player.rect()

            # coins collection (or magnet)
            for c in coins:
                if c.collected: continue
                if magnet_active and (abs(c.x - player.x) < 90) and (abs(c.y - player.y) < 120):
                    # auto-collect
                    c.collected = True
                    coin_count += 1
                elif p_rect.colliderect(c.rect()):
                    c.collected = True
                    coin_count += 1

            # powerups pickup
            for pu in powerups:
                if p_rect.colliderect(pu.rect()):
                    if pu.kind == 'magnet':
                        magnet_active = True
                        magnet_timer = MAGNET_DURATION
                    elif pu.kind == 'jetpack':
                        jetpack_active = True
                        jetpack_timer = JETPACK_DURATION
                        player.on_ground = False
                        player.vy = player.jump_strength * 0.7  # small lift
                    elif pu.kind == 'sneakers':
                        sneakers_active = True
                        sneakers_timer = SNEAKERS_DURATION
                        player.jump_strength = -22
                    elif pu.kind == 'mult':
                        multiplier = 2
                        multiplier_timer = MULTIPLIER_DURATION
                    try:
                        powerups.remove(pu)
                    except:
                        pass

            # obstacle collision
            for ob in obstacles:
                if p_rect.colliderect(ob.rect()):
                    if player.hoverboard:
                        # consume hoverboard
                        player.hoverboard = False
                        try:
                            obstacles.remove(ob)
                        except:
                            pass
                    else:
                        # game over
                        game_over = True
                        save_highscore(score)
                        if score > highscore:
                            highscore = score
                        break

        # cleanup
        obstacles = [o for o in obstacles if o not in remove_obstacles]
        coins = [c for c in coins if not c.collected and c.y < SCREEN_HEIGHT + 200]
        powerups = [p for p in powerups if p.y < SCREEN_HEIGHT + 200]

        # timers update
        if magnet_active:
            magnet_timer -= dt
            if magnet_timer <= 0:
                magnet_active = False
        if jetpack_active:
            jetpack_timer -= dt
            if jetpack_timer <= 0:
                jetpack_active = False
        if sneakers_active:
            sneakers_timer -= dt
            if sneakers_timer <= 0:
                sneakers_active = False
                player.jump_strength = -18
        if multiplier_timer > 0:
            multiplier_timer -= dt
            if multiplier_timer <= 0:
                multiplier = 1

        # update player
        player.update(dt)

        # ---------- DRAW ----------
        screen.fill((135,206,235))  # sky blue

        # draw ground / tracks
        pygame.draw.rect(screen, (100,100,100), (0, GROUND_Y + PLAYER_HEIGHT // 2, SCREEN_WIDTH, SCREEN_HEIGHT - (GROUND_Y + PLAYER_HEIGHT // 2)))
        # lane lines
        for i in range(1, LANE_COUNT):
            lx = (LANE_X[i-1] + LANE_X[i]) / 2
            pygame.draw.line(screen, (180,180,180), (lx, GROUND_Y + 30), (lx, SCREEN_HEIGHT), 4)

        # draw obstacles / coins / powerups
        for ob in obstacles:
            ob.draw(screen)
        for c in coins:
            c.draw(screen)
        for p in powerups:
            p.draw(screen)

        # draw player (hoverboard if active)
        pr = player.rect()
        pygame.draw.rect(screen, player.color, pr)
        if player.hoverboard:
            hb_rect = pygame.Rect(pr.x - 6, pr.y + pr.height, pr.width + 12, 10)
            pygame.draw.rect(screen, (255,140,0), hb_rect)

        # UI
        txt_score = font.render(f"Score: {score}", True, (0,0,0))
        txt_coins = font.render(f"Coins: {coin_count}", True, (0,0,0))
        txt_dist = font.render(f"Dist: {int(distance)}", True, (0,0,0))
        txt_keys = font.render(f"Keys: {key_count}", True, (0,0,0))
        txt_high = font.render(f"High: {highscore}", True, (0,0,0))

        screen.blit(txt_score, (12, 12))
        screen.blit(txt_coins, (12, 36))
        screen.blit(txt_dist, (12, 60))
        screen.blit(txt_keys, (12, 84))
        screen.blit(txt_high, (SCREEN_WIDTH - 120, 12))

        if magnet_active:
            screen.blit(font.render("Magnet", True, (0,0,0)), (SCREEN_WIDTH - 120, 36))
        if jetpack_active:
            screen.blit(font.render("Jetpack", True, (0,0,0)), (SCREEN_WIDTH - 120, 60))
        if player.hoverboard:
            screen.blit(font.render("Hoverboard", True, (0,0,0)), (SCREEN_WIDTH - 120, 84))

        if game_over:
            go_font = pygame.font.SysFont("Arial", 36)
            go = go_font.render("GAME OVER", True, (255,0,0))
            info = font.render("Press R to Revive (uses a key) or Q to Quit", True, (0,0,0))
            screen.blit(go, (SCREEN_WIDTH/2 - go.get_width()/2, SCREEN_HEIGHT/2 - 60))
            screen.blit(info, (SCREEN_WIDTH/2 - info.get_width()/2, SCREEN_HEIGHT/2))

        pygame.display.flip()

    pygame.quit()

if _name_ == "_main_":
    # If this file is run directly it launches the game.
    # But when saved to /mnt/data by this script, the user can download and run on their machine / Replit.
    main()

ModuleNotFoundError: No module named 'pygame'